In [0]:
OBS_PATH = '/datascope/subaru/data/cmdfit/dSph/umi_tpall3e_g24.cat'

In [0]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from scipy.special import logsumexp
from scipy.interpolate import interp1d

from astropy import wcs
from astropy import units as u
from astropy.coordinates import Angle

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
%load_ext autoreload

In [0]:
%autoreload 2

# Load observations

In [0]:
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color

In [0]:
obs = SubaruHSC.text_observation_reader().read(OBS_PATH)
obs.data.head()

# Plot color-magnitude diagrams

In [0]:
hsc = SubaruHSC.photometry()
cmd = CMD([
        ColorAxis(
            Color([hsc.magnitudes['g'], hsc.magnitudes['i']]),
            limits=(-1, 4)),
        MagnitudeAxis(
            hsc.magnitudes['g'],
            limits=(15.5, 24.5)
        )
    ])
ccd = CCD([
    cmd.axes[0],
    ColorAxis(
        Color([hsc.magnitudes['g'], hsc.magnitudes['nb515']]),
        limits=(-0.5, 0.5)
    )
])

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_observation(axs[0], obs)
ccd.plot_observation(axs[1], obs)

f.tight_layout()

# Plot field of view of instrument

In [0]:
from pfs.ga.targeting.projection import Pointing, WcsProjection
from pfs.ga.targeting.instrument import SubaruWFC, SubaruPFI
from pfs.ga.targeting.diagram import FOV, FP

In [0]:
pointing = Pointing(Angle('15h 09m 08.5s').degree, Angle('+67d 13m 21s').degree)
print(pointing.ra, pointing.dec)
wcs = WcsProjection(pointing, proj='TAN')
wfc = SubaruWFC(wcs.pointing)
fov = FOV(projection=wcs)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=120, subplot_kw=dict(projection=fov.projection.wcs))
fov.plot_catalog(ax, obs)

for pp in [(228.2, 67.5, 0),
           (226.3, 67.5, 0),
           (226.0, 66.9, 0),
           (228.1, 66.955, 40)]:
    wfc = SubaruWFC(Pointing(*pp))
    pfi = SubaruPFI(wfc)
    pfi.plot_focal_plane(ax, fov, blocks=True, c='b', ls='-')
    pfi.plot_focal_plane(ax, fov, corners=True, c='r', ls='-')

# Plot focal plane

In [0]:
f, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=240)

fp = FP()
wfc = SubaruWFC(Pointing(228.2, 67.5, 0))
pfi = SubaruPFI(wfc)
pfi.plot_focal_plane(ax, fp, blocks=True, c='b', ls='-')
pfi.plot_cobras(ax, fp)

ax.set_xlim(-250, 250)
ax.set_ylim(-250, 250)

In [0]:
wfc = SubaruWFC(Pointing(228.2, 67.5, 0))
pfi = SubaruPFI(wfc)
centers, radec = pfi.get_cobra_centers()
radec.shape

In [0]:
f, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=120)

r = np.random.rand(radec.shape[0])

fp = FP()
wfc = SubaruWFC(Pointing(228.2, 67.5, 0))
pfi = SubaruPFI(wfc)
pfi.plot_focal_plane(ax, fp, outline=True, c='b', ls='-')
pfi.plot_cobras(ax, fp, r)

ax.set_xlim(-250, 250)
ax.set_ylim(-250, 250)